In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.svm import LinearSVC

from scipy.optimize import minimize

%matplotlib inline

In [2]:
df = pd.read_csv(r'heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [6]:
df['ChestPainType'].value_counts()

ChestPainType
ASY    496
NAP    203
ATA    173
TA      46
Name: count, dtype: int64

In [7]:
df['Sex'].value_counts()

Sex
M    725
F    193
Name: count, dtype: int64

In [8]:
df['RestingECG'].value_counts()

RestingECG
Normal    552
LVH       188
ST        178
Name: count, dtype: int64

In [9]:
df['ExerciseAngina'].value_counts()

ExerciseAngina
N    547
Y    371
Name: count, dtype: int64

In [10]:
df['ST_Slope'].value_counts()

ST_Slope
Flat    460
Up      395
Down     63
Name: count, dtype: int64

In [11]:
# Separate features and target
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

In [12]:
# Identify numerical and categorical columns
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
label_encoded_features = ['Sex', 'ExerciseAngina']
one_hot_encoded_features = ['ChestPainType', 'RestingECG', 'ST_Slope']

In [13]:
# Define the transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Scale numerical features
        ('label', OrdinalEncoder(), label_encoded_features),  # Use OrdinalEncoder for binary categorical features
        ('onehot', OneHotEncoder(drop='first'), one_hot_encoded_features)  # One-hot encode other categorical features
    ])

# Apply the transformations
X_transformed = preprocessor.fit_transform(X)

# Convert to DataFrame to see the final result
# Retrieve feature names after transformation for readability
onehot_encoded_columns = preprocessor.named_transformers_['onehot'].get_feature_names_out(one_hot_encoded_features)
column_names = numerical_features + label_encoded_features + list(onehot_encoded_columns)
X_transformed = pd.DataFrame(X_transformed, columns=column_names)

# Display the preprocessed data
X_transformed.head()

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex,ExerciseAngina,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ST_Slope_Flat,ST_Slope_Up
0,-1.433140,0.410909,0.825070,1.382928,-0.832432,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.478484,1.491752,-0.171961,0.754157,0.105664,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,-1.751359,-0.129513,0.770188,-1.525138,-0.832432,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,-0.584556,0.302825,0.139040,-1.132156,0.574711,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.051881,0.951331,-0.034755,-0.581981,-0.832432,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [14]:
X_transformed.shape

(918, 14)

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, 
                                                    test_size=0.20, 
                                                    random_state=47)

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((734, 14), (184, 14), (734,), (184,))

In [ ]:

class SVM:
    
    def __init__(self, C=1.0):
        self.C = C  # Regularization parameter
        self.w = None
        self.b = None
        self.alphas = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.support_alphas = None
    
    def fit(self, X, y):
        # Convert y from {0, 1} to {-1, 1} if needed
        y = np.where(y == 0, -1, y)
        
        # Store the training data
        self.X = X
        self.y = y
        m, n = X.shape
        
        # Compute the gram matrix
        K = X @ X.T
        
        # Objective function: Maximize f(alpha) = 1^T @ alpha - 0.5 * (alpha * y)^T @ K @ (alpha * y)
        def objective(alpha):
            alpha_y = alpha * y  # Element-wise product of alpha and y
            # Objective function to minimize (converted from maximization form)
            return -np.sum(alpha) + 0.5 * np.dot(alpha_y, K @ alpha_y)
        
        # Equality constraint: alpha^T * y = 0
        constraints = [
            {'type': 'eq', 'fun': lambda alpha: np.dot(y, alpha)}
        ]

        # Bounds for each alpha: 0 <= alpha <= C
        bounds = [(0, self.C) for _ in range(m)]
        
        # Initial guess for alpha
        alpha0 = np.zeros(m)
        
        # Solve the problem
        result = minimize(objective, alpha0, bounds=bounds, constraints=constraints, method='SLSQP')
        self.alphas = result.x
        
        # Extract support vectors where 0 < alpha <= C
        epsilon = 1e-5
        support_vector_indices = np.where((self.alphas > epsilon) & (self.alphas < self.C))[0]
        margin_violator_indices = np.where(self.alphas == self.C)[0]
        
        # Store support vectors and margin violators
        self.support_vectors = X[support_vector_indices]
        self.support_vector_labels = y[support_vector_indices]
        self.support_alphas = self.alphas[support_vector_indices]
        self.margin_violators = X[margin_violator_indices]
        
        # Compute w
        self.w = self.X.T @ (self.alphas * self.y)
        
        # Compute b using any support vector
        self.b = np.mean(
            [self.support_vector_labels[i] - np.dot(self.w, self.support_vectors[i]) 
                for i in range(len(self.support_alphas))]
        )
        
    def predict(self, X):
        # Make predictions based on the sign of the decision function
        return np.sign(np.dot(X, self.w) + self.b)
    
    def decision_function(self, X):
        # Compute the decision function values
        return np.dot(X, self.w) + self.b

In [61]:
model = SVM(C=100.0)
model.fit(X_train.values, y_train.values)

In [62]:
model.w, model.b

(array([ 0.04846917,  0.08573436, -0.43564107,  0.01065142,  0.41590359,
         1.27832979,  0.47764243, -1.5377542 , -1.21309636, -0.96151302,
        -0.20718278,  0.12469867,  1.27276447, -0.5273545 ]),
 -0.6745349290705213)

In [63]:
y_pred_train = model.predict(X_train.values)
y_pred_train = np.where(y_pred_train == -1, 0, 1)
y_pred_test = model.predict(X_test.values)
y_pred_test = np.where(y_pred_test == -1, 0, 1)

In [55]:
from sklearn.metrics import classification_report

In [64]:
# training set
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.87      0.84      0.86       328
           1       0.88      0.90      0.89       406

    accuracy                           0.88       734
   macro avg       0.88      0.87      0.87       734
weighted avg       0.88      0.88      0.88       734



In [65]:
# test set
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.86      0.78      0.82        82
           1       0.84      0.90      0.87       102

    accuracy                           0.85       184
   macro avg       0.85      0.84      0.84       184
weighted avg       0.85      0.85      0.85       184



In [66]:
model.support_vectors.shape

(46, 14)

In [67]:
# either mis-classified or lies within the margin but are still on the correct side of the decision boundary
model.margin_violators.shape

(209, 14)

In [46]:
# model.alphas

#### **Using `sklearn.svm` `LinearSVC`**

In [92]:
model1 = LinearSVC(
    dual=False,
    C=100.0,
    max_iter=10000
)

model1.fit(X_train, y_train)

LinearSVC(C=100.0, dual=False, max_iter=10000)

In [93]:
y_train_pred1 = model1.predict(X_train)
y_test_pred1 = model1.predict(X_test)

In [94]:
# train set
print(classification_report(y_train, y_train_pred1))

              precision    recall  f1-score   support

           0       0.87      0.84      0.86       328
           1       0.88      0.90      0.89       406

    accuracy                           0.87       734
   macro avg       0.87      0.87      0.87       734
weighted avg       0.87      0.87      0.87       734



In [98]:
# test set
print(classification_report(y_test, y_test_pred1))

              precision    recall  f1-score   support

           0       0.84      0.79      0.82        82
           1       0.84      0.88      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184

